In [48]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [49]:
# Load the dataset
data = pd.read_csv('/content/heart (1).csv')


In [50]:
# Handling Missing Values
# Identify columns with missing values
columns_with_missing = data.columns[data.isnull().any()].tolist()

if columns_with_missing:
    # Impute missing values with the mean
    imputer = SimpleImputer(strategy='mean')
    data[columns_with_missing] = imputer.fit_transform(data[columns_with_missing])


In [51]:

# Categorical Encoding
categorical_columns = ['categorical_column1', 'categorical_column2']

missing_columns = list(set(categorical_columns) - set(data.columns))
if missing_columns:
    print(f"Columns {missing_columns} not found in the dataset.")
else:
    # Perform one-hot encoding
    one_hot_encoder = OneHotEncoder(sparse=False, drop='first')
    encoded_data = pd.DataFrame(one_hot_encoder.fit_transform(data[categorical_columns]))
    # Concatenate the encoded data with the original dataset
    data_encoded = pd.concat([data, encoded_data], axis=1)
    # Drop the original categorical columns
    data_encoded.drop(columns=categorical_columns, inplace=True)

    # Feature Scaling
    columns_to_scale = ['numeric_column1', 'numeric_column2']
    # Perform standardization
    scaler = StandardScaler()
    data_encoded[columns_to_scale] = scaler.fit_transform(data_encoded[columns_to_scale])

    # View the preprocessed dataset
    print(data_encoded.head())


Columns ['categorical_column1', 'categorical_column2'] not found in the dataset.


In [52]:


# Handling Missing Values
columns_with_missing = data.columns[data.isnull().any()].tolist()

if columns_with_missing:
    # Impute missing values with the mean
    imputer = SimpleImputer(strategy='mean')
    data[columns_with_missing] = imputer.fit_transform(data[columns_with_missing])

# Categorical Encoding
categorical_columns = ['categorical_column1', 'categorical_column2']

missing_columns = list(set(categorical_columns) - set(data.columns))
if missing_columns:
    print(f"Columns {missing_columns} not found in the dataset.")
else:
    # Perform one-hot encoding
    one_hot_encoder = OneHotEncoder(sparse=False, drop='first')
    encoded_data = pd.DataFrame(one_hot_encoder.fit_transform(data[categorical_columns]))
    # Concatenate the encoded data with the original dataset
    data_encoded = pd.concat([data, encoded_data], axis=1)
    # Drop the original categorical columns
    data_encoded.drop(columns=categorical_columns, inplace=True)

    # Feature Scaling
    columns_to_scale = ['numeric_column1', 'numeric_column2']
    # Perform standardization
    scaler = StandardScaler()
    data_encoded[columns_to_scale] = scaler.fit_transform(data_encoded[columns_to_scale])

    # Handling Outliers (Example: Replacing outliers with median)
    data_encoded[columns_to_scale] = data_encoded[columns_to_scale].apply(
        lambda x: x.mask(x.abs() > 3 * x.std(), x.median()), axis=0
    )

    # Dimensionality Reduction (Example: Using PCA to reduce dimensionality to 2)
    pca = PCA(n_components=2)
    data_reduced = pca.fit_transform(data_encoded)

    # Data Normalization (Example: Min-Max Scaling)
    min_max_scaler = MinMaxScaler()
    data_normalized = min_max_scaler.fit_transform(data_reduced)

    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(
        data_normalized, target_column, test_size=0.2, random_state=42
    )

    # View the preprocessed dataset
    print(X_train.shape, X_test.shape)


Columns ['categorical_column1', 'categorical_column2'] not found in the dataset.


In [53]:
print(data.columns)

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')


In [54]:
print(data.head())

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   3       145   233    1        0      150      0      2.3      0   
1   37    1   2       130   250    0        1      187      0      3.5      0   
2   41    0   1       130   204    0        0      172      0      1.4      2   
3   56    1   1       120   236    0        1      178      0      0.8      2   
4   57    0   0       120   354    0        1      163      1      0.6      2   

   ca  thal  target  
0   0     1       1  
1   0     2       1  
2   0     2       1  
3   0     2       1  
4   0     2       1  


In [55]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import PolynomialFeatures


# Separate the features (X) and target variable (y)
X = data.drop(columns=['target'])
y = data['target']

# Feature Selection with SelectKBest
k = 10  # Number of top features to select
selector = SelectKBest(score_func=f_regression, k=k)
X_selected = selector.fit_transform(X, y)
selected_feature_indices = selector.get_support(indices=True)
selected_features = X.columns[selected_feature_indices]

# Create Interaction Features
interaction_features = pd.DataFrame()
for i in range(len(selected_features)):
    for j in range(i + 1, len(selected_features)):
        interaction_feature = X[selected_features[i]] * X[selected_features[j]]
        interaction_features[f'{selected_features[i]}_{selected_features[j]}'] = interaction_feature

# Create Polynomial Features
degree = 2  # Degree of polynomial features
poly_features = PolynomialFeatures(degree=degree, include_bias=False)
X_poly = poly_features.fit_transform(X[selected_features])

# Combine selected features, interaction features, and polynomial features
X_final = pd.concat([X[selected_features], interaction_features, pd.DataFrame(X_poly)], axis=1)

# View the selected and engineered features
print(X_final.head())


   age  sex  cp  trestbps  thalach  exang  oldpeak  slope  ca  thal  ...  \
0   63    1   3       145      150      0      2.3      0   0     1  ...   
1   37    1   2       130      187      0      3.5      0   0     2  ...   
2   41    0   1       130      172      0      1.4      2   0     2  ...   
3   56    1   1       120      178      0      0.8      2   0     2  ...   
4   57    0   0       120      163      1      0.6      2   0     2  ...   

      55   56   57   58   59   60   61   62   63   64  
0   5.29  0.0  0.0  2.3  0.0  0.0  0.0  0.0  0.0  1.0  
1  12.25  0.0  0.0  7.0  0.0  0.0  0.0  0.0  0.0  4.0  
2   1.96  2.8  0.0  2.8  4.0  0.0  4.0  0.0  0.0  4.0  
3   0.64  1.6  0.0  1.6  4.0  0.0  4.0  0.0  0.0  4.0  
4   0.36  1.2  0.0  1.2  4.0  0.0  4.0  0.0  0.0  4.0  

[5 rows x 120 columns]


In [56]:

# Separate the features (X) and target variable (y)
X = data.drop(columns=['target'])
y = data['target']

# Feature Selection with SelectKBest
k = 10  # Number of top features to select
selector = SelectKBest(score_func=f_regression, k=k)
X_selected = selector.fit_transform(X, y)
selected_feature_indices = selector.get_support(indices=True)
selected_features = X.columns[selected_feature_indices]

# Create Interaction Features
interaction_features = pd.DataFrame()
for i in range(len(selected_features)):
    for j in range(i + 1, len(selected_features)):
        interaction_feature = X[selected_features[i]] * X[selected_features[j]]
        interaction_features[f'{selected_features[i]}_{selected_features[j]}'] = interaction_feature

# Create Polynomial Features
degree = 2  # Degree of polynomial features
poly_features = PolynomialFeatures(degree=degree, include_bias=False)
X_poly = poly_features.fit_transform(X[selected_features])

# Combine selected features, interaction features, and polynomial features
X_final = pd.concat([X[selected_features], interaction_features, pd.DataFrame(X_poly)], axis=1)

# View the selected and engineered features
print(X_final.head())


   age  sex  cp  trestbps  thalach  exang  oldpeak  slope  ca  thal  ...  \
0   63    1   3       145      150      0      2.3      0   0     1  ...   
1   37    1   2       130      187      0      3.5      0   0     2  ...   
2   41    0   1       130      172      0      1.4      2   0     2  ...   
3   56    1   1       120      178      0      0.8      2   0     2  ...   
4   57    0   0       120      163      1      0.6      2   0     2  ...   

      55   56   57   58   59   60   61   62   63   64  
0   5.29  0.0  0.0  2.3  0.0  0.0  0.0  0.0  0.0  1.0  
1  12.25  0.0  0.0  7.0  0.0  0.0  0.0  0.0  0.0  4.0  
2   1.96  2.8  0.0  2.8  4.0  0.0  4.0  0.0  0.0  4.0  
3   0.64  1.6  0.0  1.6  4.0  0.0  4.0  0.0  0.0  4.0  
4   0.36  1.2  0.0  1.2  4.0  0.0  4.0  0.0  0.0  4.0  

[5 rows x 120 columns]


In [57]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error


In [58]:
print(data.columns)

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')


In [59]:
# Separate the features (X) and target variable (y)
X = data.drop(columns=['target'])
y = data['target']



In [60]:

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [61]:
# Random Forest
random_forest = RandomForestRegressor(random_state=42)
random_forest.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [62]:
# Predict on the test set
y_pred_rf = random_forest.predict(X_test)

In [63]:
# Evaluate Random Forest
mse_rf = mean_squared_error(y_test, y_pred_rf)
print("Random Forest Mean Squared Error:", mse_rf)

# Evaluate Random Forest
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest:")
print("Mean Squared Error:", mse_rf)
print("R-squared:", r2_rf)

Random Forest Mean Squared Error: 0.11720327868852458
Random Forest:
Mean Squared Error: 0.11720327868852458
R-squared: 0.5300502155172415


In [64]:
# Gradient Boosting
gradient_boosting = GradientBoostingRegressor(random_state=42)
gradient_boosting.fit(X_train, y_train)

GradientBoostingRegressor(random_state=42)

In [65]:
# Predict on the test set
y_pred_gb = gradient_boosting.predict(X_test)

In [66]:
# Evaluate Gradient Boosting
mse_gb = mean_squared_error(y_test, y_pred_gb)
print("Gradient Boosting Mean Squared Error:", mse_gb)

Gradient Boosting Mean Squared Error: 0.13350601821956246


In [67]:
# Hyperparameter tuning with GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001]
}

grid_search = GridSearchCV(GradientBoostingRegressor(random_state=42), param_grid, cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(random_state=42),
             param_grid={'learning_rate': [0.1, 0.01, 0.001],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 200, 300]})

In [68]:
# Get the best hyperparameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 300}


In [69]:

# Predict on the test set with the best model
y_pred_best = best_model.predict(X_test)

In [70]:

# Evaluate the best model
mse_best = mean_squared_error(y_test, y_pred_best)
print("Best Model Mean Squared Error:", mse_best)

Best Model Mean Squared Error: 0.11960904551240514


In [71]:

# Train Gradient Boosting
gradient_boosting = GradientBoostingRegressor(random_state=42)
gradient_boosting.fit(X_train, y_train)


GradientBoostingRegressor(random_state=42)

In [72]:
# Predict on the test set using Gradient Boosting
y_pred_gb = gradient_boosting.predict(X_test)

In [73]:
# Evaluate Gradient Boosting
mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

print("\nGradient Boosting:")
print("Mean Squared Error:", mse_gb)
print("R-squared:", r2_gb)


Gradient Boosting:
Mean Squared Error: 0.13350601821956246
R-squared: 0.46468114892781054


In [74]:

# Compare and analyze the results
if mse_rf < mse_gb:
    print("\nRandom Forest performs better.")
elif mse_gb < mse_rf:
    print("\nGradient Boosting performs better.")
else:
    print("\nBoth models perform equally.")

if r2_rf > r2_gb:
    print("Random Forest has a higher R-squared.")
elif r2_gb > r2_rf:
    print("Gradient Boosting has a higher R-squared.")
else:
    print("Both models have the same R-squared.")



Random Forest performs better.
Random Forest has a higher R-squared.
